In [1]:
#! usr/bin/env python3
from tqdm import tqdm_notebook as tqdm
import praw
import pandas as pd
import datetime as dt
from PIL import Image
import urllib.request
import wikipedia
import feather

import re
import glob
import time

import ssl

ssl._create_default_https_context = ssl._create_unverified_context

import IPython.display as ipd
import pyttsx3
import os
import uuid
import operator

from shutil import copyfile

from pydub import AudioSegment
from google.cloud import texttospeech

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/anthonycelio/xx/My_Projectectect-ef53cc7c18c4.json"

In [ ]:
reddit = praw.Reddit(client_id='ZWkdQHV89SHfEw', \
                     client_secret='DE9g8daIEcx7SEH3seN41aDKmIs', \
                     user_agent='Chester', \
                     username='user_00000000000001', \
                     password='Russianriver987')

subreddit = reddit.subreddit('Unexpected')
#subreddit = reddit.subreddit('cringeanarchy')

#top_subreddit = subreddit.hot()
top_subreddit = subreddit.top('week')

#top_subreddit = subreddit.top(limit=50)
#for submission in subreddit.hot(limit=1):
#    print(submission.title, submission.id)
    
topics_dict = { "title":[],
               "score":[],
               "id":[],
               "url":[],
               "comms_num":[],
               "created": [],
               "body":[],
               "vid":[]
                  }

for submission in top_subreddit:
    topics_dict["title"].append(submission.title)
    topics_dict["score"].append(submission.score)
    topics_dict["id"].append(submission.id)
    topics_dict["url"].append(submission.url)
    topics_dict["comms_num"].append(submission.num_comments)
    topics_dict["created"].append(submission.created)
    topics_dict["body"].append(submission.selftext)
    topics_dict["vid"].append(submission.media)
    
topics_data = pd.DataFrame(topics_dict)

In [ ]:
#submission.media['reddit_video']['fallback_url']

In [ ]:
topics_data

In [ ]:
topics_data['comms_num'][5]

In [ ]:
brk = 0
use = []
for i in range(100):
    if topics_data["comms_num"][i] >= 13:
        if topics_data["vid"][i] != None:
            try:
                xex = topics_data["vid"][i]["reddit_video"]["duration"]
                if xex > 4 and xex < 30:
                    use.append(i)
                    brk += 1
            except:
                pass
    
    #if brk >= 5:
     #   break

In [ ]:
len(use)

In [ ]:
z99 = 0
for z, i in enumerate(topics_data['url']):
    if z99 < 10:
        if i[-3:] == 'jpg':
            urllib.request.urlretrieve(i, i[-10:])
            z99 += 1
            print(z, i)

In [ ]:
topics_data.loc[0]

In [ ]:
!pwd

In [ ]:
def ret_com(num):
    po = topics_data['id'][num]
    submission = reddit.submission(id=po)
    submission.comments.replace_more(limit=1)
    chop = []
    
    for top_level_comment in submission.comments:
        chop.append([top_level_comment.body])
        for i in (top_level_comment.replies):
            chop.append([i.body])
            for x in (i.replies):
                chop.append([x.body])
                for z in (x.replies):
                    chop.append([z.body])
                    for cc in (z.replies):
                        chop.append([cc.body])
                        for wo in (cc.replies):
                            chop.append([wo.body])
                            for zoo in (wo.replies):
                                chop.append([zoo.body])

    grab = []
    for i in chop:
        if len(i[0]) > 10:
            if len(i[0]) < 300:
                for x in i:
                    i = re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', x)
                    i = re.sub(r'[^a-zA-Z1-9.,()\'"?! ]+', '', i)
                    i = i.replace('ITAP', 'I took a picture')

                    grab.append(i)    
    #[topics_data['title'][num]] + grab
    frst = topics_data['title'][num].replace('ITAP of ', '')
    grab.insert(0, frst.replace('ITAP ', ''))

    return grab[:24]

In [ ]:
cc = ret_com(1)

In [ ]:
cc

In [ ]:
len(cc)

In [ ]:
if cc[-1:][0][-1:] == '?':
    cc = cc[:-1]
if cc[-1:][0][-2:] == '?':
    cc = cc[:-1]

In [ ]:
cc[:-1]

In [ ]:
def TTS_lena(str, wave_name):
    client = texttospeech.TextToSpeechClient()
    synthesis_input = texttospeech.types.SynthesisInput(text=str)

    voice = texttospeech.types.VoiceSelectionParams(
        language_code='en-AU',
        name='en-AU-Wavenet-A',
        ssml_gender=texttospeech.enums.SsmlVoiceGender.FEMALE)

    audio_config = texttospeech.types.AudioConfig(
        speaking_rate=0.91,
        pitch=0,
        audio_encoding=texttospeech.enums.AudioEncoding.MP3)
    response = client.synthesize_speech(synthesis_input, voice, audio_config)
    with open('output.mp3', 'wb') as out:
        out.write(response.audio_content)
    newAudio = AudioSegment.from_mp3('output.mp3')
    newAudio.export(wave_name + '.wav', format="wav")
    os.remove('output.mp3')

In [3]:
def TTS_stac(str, wave_name):
    client = texttospeech.TextToSpeechClient()
    synthesis_input = texttospeech.types.SynthesisInput(text=str)

    voice = texttospeech.types.VoiceSelectionParams(
        language_code='en-US',
        name='en-US-Wavenet-F',
        ssml_gender=texttospeech.enums.SsmlVoiceGender.FEMALE)

    audio_config = texttospeech.types.AudioConfig(
        speaking_rate=0.89,
        pitch=-1,
        audio_encoding=texttospeech.enums.AudioEncoding.MP3)
    response = client.synthesize_speech(synthesis_input, voice, audio_config)
    with open('output.mp3', 'wb') as out:
        out.write(response.audio_content)
    newAudio = AudioSegment.from_mp3('output.mp3')
    newAudio.export(wave_name + '.wav', format="wav")
    os.remove('output.mp3')

In [ ]:
def wav_2_phone(wav_file):
    new = AudioSegment.from_wav(wav_file)
    new = new.apply_gain(12).low_pass_filter(30000).high_pass_filter(30000)
    new = new + 15
    new.export(wav_file, format="wav")

In [ ]:
wdir = '/Users/anthonycelio/Documents/deep_news/!machine/corespondants/16/'
if not os.path.exists(wdir): os.makedirs(wdir)

In [ ]:
(use[10:14])

In [ ]:
for m in tqdm(range(len(use[10:15]))):
    t = str(int(time.time()))
    sha = wdir+t+'/'
    os.makedirs(sha)
    os.chdir(sha)

    cc = ret_com(use[10:15][m])[:20]
    if cc[-1:][0][-1:] == '?':
        cc = cc[:-1]
    if cc[-1:][0][-2:] == '?':
        cc = cc[:-1]
    mp4s = topics_data["vid"][use[10:15][m]]["reddit_video"]["fallback_url"]
    urllib.request.urlretrieve(mp4s, mp4s[-14:]+'.mp4')
    for z, i in enumerate(tqdm(cc)):
        if z % 2 == 0:
            TTS_lena(i, (str(z)).zfill(2))
            wav_2_phone((str(z)).zfill(2)+'.wav')
        else:
            TTS_stac(i, (str(z)).zfill(2))

In [ ]:
su = '''The Parasol (also known as El Quitasol) is one of a cartoon series of oil on linen paintings made by the painter Francisco Goya. This series of paintings was specifically made in order to be transformed into tapestries that would be hung on the walls of the Royal Palace of El Pardo in Madrid, Spain. The tapestries showed serene events in everyday life, which made them a nice addition to the dining room of Prince and Princess of Asturias—the future King Charles IV and Maria Luisa of Parma. The queen called on Goya because she wanted to decorate the dining room with cheerful scenes; The Parasol and the other tapestry paintings were Goya's response to this request. The painting is currently located in the Museo del Prado in Madrid as is another in the series, Blind man's bluff. '''

In [ ]:
TTS_stac(su, '/Users/anthonycelio/Desktop/goya')

In [ ]:
!pip install feather-format

In [15]:
wdir = '/Users/anthonycelio/Documents/deep_news/!machine/art_set/wiki_paintings/1a5a22f954a9/'
os.chdir(wdir)

In [16]:
path = wdir+'my_data.feather'
wdf = feather.read_dataframe(path)
wdf.loc[0][0]

"Paris Street; Rainy Day (French: Rue de Paris, temps de pluie) is a large 1877 oil painting by the French artist Gustave Caillebotte (1848–1894), and is his best known work It shows a number of individuals walking through the Place de Dublin, then known as the Carrefour de Moscou, at an intersection to the east of the Gare Saint-Lazare in north Paris Although Caillebotte was a friend and patron of many of the impressionist painters, and this work is part of that school, it differs in its realism and reliance on line rather than broad brush strokesCaillebotte's interest in photography is evident The figures in the foreground appear out of focus, those in the mid-distance (the carriage and the pedestrians in the intersection) have sharp edges, while the features in the background become progressively indistinct The severe cropping of some figures –particularly the man to the far right– further suggests the influence of photographyThe painting was first shown at the Third Impressionist E

In [17]:
TTS_stac(wdf.loc[0][0], 'TTS')

In [ ]:
sea = [int(i) for i in os.listdir(wdir) if len(i) is 10]

In [ ]:
sea

In [ ]:
print('''var vs = '''+'"'+str(wdir)+'"'+'''
var myblocnFolder = new Folder(vs).getFiles()
lena = '''+str(sea))

In [ ]:
wav_2_phone((str(z)).zfill(2)+'.wav')

In [ ]:
lena = '/Users/anthonycelio/Documents/deep_news/correspondants/lena/'

In [ ]:
os.chdir(lena)

In [ ]:
TTS_stac('Welcome back. Joining us now on the phone, our correspondant Lena Bronsky. How are you, Lena?', lena+'int_lena1')

In [ ]:
TTS_lena('Fine Stacy, How are you?', lena+'lena_r1')
wav_2_phone(lena+'lena_r1'+'.wav')

In [ ]:
TTS_stac('''Great. I understand you have some photos for us. What's your first image?''', lena+'St_go_image')

In [ ]:
TTS_stac('''Okay, What's next, Lena?''', lena+'St_nextQ')

In [ ]:
TTS_stac('''and this concludes our broadcast. Thanks for watching.''', lena+'St_show_end' )

In [ ]:
//*[@id="content"]/article/div[4]/div[1]/p/text()

In [ ]:
TTS_lena('Thank you for having me, Stacy.', lena+'lena_out')
wav_2_phone(lena+'lena_out'+'.wav')

In [ ]:
print (wikipedia.summary("Lucca Madonna"))
len(wikipedia.summary("Lucca Madonna"))

In [ ]:
tit = "The Accolade painting)"

In [ ]:
tit = "Lucca Madonna"

In [ ]:
tit = "Emily Carr (1939) Odds and Ends"

In [ ]:
print (wikipedia.summary(tit))
len(wikipedia.summary(tit))

In [ ]:
print (wikipedia.summary(tit))
len(wikipedia.summary(tit))

In [ ]:
wikipedia.page(tit).content.replace("\"", "")

In [ ]:
(wikipedia.page(imago).summary)

In [ ]:
zep = wikipedia.page(imago)

In [ ]:
zep.content.rsplit('\n\n\n== Ndfotes')[0]

In [ ]:
zo = zep.content.replace('\n\n\n== Description ==\n', ' ')

zo = zo.replace("\n\n\n== History ==", ' ')
zo = zo.replace("\n\n\n== See also ==\n", ' ')
zo = zo.rsplit('\n\n\n== Notes')[0]
zo = zo.replace("\"", "")
zo = zo.replace("\n", "")

In [ ]:
zo

In [ ]:
len(zo)

In [ ]:
und = 0
bruh = ""
if len(zo) > 1500:
    for i in zo.rsplit('.'):
        if und < 1500:
            bruh += i
            und += len(i)
    bruh += '.'

In [ ]:
len(bruh)

In [ ]:
und 

In [ ]:
bruh

In [ ]:
wdir = '/Users/anthonycelio/Documents/deep_news/!machine/art_set/'

In [ ]:
t = str(uuid.uuid4())[-12:]
sha = wdir+t+'/'
os.makedirs(sha)
os.chdir(sha)

for m in tqdm(zep.images):
    
    urllib.request.urlretrieve(m, sha+m[-14:])

In [ ]:
d = {}
for i in os.listdir(sha):
    cu = os.stat(i).st_size
    d[i] = cu
    
for x in os.listdir(sha):
    if x != max(d.items(), key=operator.itemgetter(1))[0]:
        os.remove(x)

In [ ]:
max(d.items(), key=operator.itemgetter(1))[0]

In [ ]:
d.values()

In [ ]:
import time
import requests
import urllib.request
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

In [ ]:
ma = 'https://en.wikipedia.org/wiki/Wikipedia:Featured_pictures/Artwork/Paintings'

In [ ]:
req = Request(ma, headers={'User-Agent': 'Mozilla/5.0'})
webpage = urlopen(req).read()
soup = BeautifulSoup(webpage, "html.parser")

In [ ]:
mydivs = soup.findAll("div", {"class": "gallerytext"})


In [ ]:
imago = str(mydivs[100].findAll('a')[0]).rsplit(' ')[1][12:-1]

In [ ]:
wdf = pd.DataFrame([0])

In [ ]:
wdf[0] = zo

In [ ]:
wdf.loc[1] = 'asdfasdfasdf'

In [ ]:
wdf

In [ ]:
path = wdir+'my_data.feather' 
feather.write_dataframe(wdf, path)
wdf = feather.read_dataframe(path)

In [ ]:
Image.MAX_IMAGE_PIXELS = None

In [ ]:
werdir = '/Users/anthonycelio/Documents/deep_news/!machine/art_set/bdb661269a80/'

In [ ]:
emage = '/Users/anthonycelio/Documents/deep_news/!machine/art_set/bdb661269a80/6dec0ogle_Earth.jpg'

In [ ]:
Image.MAX_IMAGE_PIXELS = None
from PIL import Image
img = Image.open(emage)

In [ ]:
# W x H
new_img = img.resize((int(0.3 * img.width), int(0.3 * img.height)))
new_img.save(werdir+"resized.jpg", "JPEG", optimize=False)